In [1]:
import numpy as np
import librosa
import glob
import os
from random import randint
import torch
import torch.nn as nn
from torch.utils import data
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import matplotlib.pyplot as plt
%matplotlib inline

import import_ipynb
from Triplet_LSTM_Net import *
from Triplet_LSTM_Dataloader import *

importing Jupyter notebook from Triplet_LSTM_Net.ipynb
importing Jupyter notebook from Triplet_LSTM_Dataloader.ipynb


In [2]:
torch.cuda.set_device(3)
print(torch.cuda.current_device())

3


In [5]:
tripleNet_LSTM_model = TripletLSTM().double().cuda()
state_dict = torch.load('./model/tripleNet_LSTM.pkl',map_location={'cuda:0':'cuda:3'})
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] 
    new_state_dict[name] = v
tripleNet_LSTM_model.load_state_dict(new_state_dict)
tripleNet_LSTM_model = tripleNet_LSTM_model.eval()

In [16]:
data_dir = "/media/data/cuixuange/ScrawlMusic/train_data_normalizatin/"
def load_exits_file(train_data,index):
    str_data = train_data[index].split(",")      
    x1_name = data_dir + str_data[0]+".npy"
    x2_name = data_dir + str_data[1]+".npy"
    x3_name = data_dir + str_data[2]+".npy"
    if os.path.isfile(x1_name) and os.path.isfile(x2_name) and os.path.isfile(x3_name): 
        return x1_name,x2_name,x3_name,index
    else:
        index += 6
        return load_exits_file(train_data,index)

result_score_dict = {}
result = []

with torch.no_grad():
    for key,token in enumerate(my_test_data):
        x1_name,x2_name,x3_name,exit_index = load_exits_file(my_test_data,key)
        x1 = torch.from_numpy(np.load(x1_name)).unsqueeze(0).unsqueeze(0).cuda()
        x2 = torch.from_numpy(np.load(x2_name)).unsqueeze(0).unsqueeze(0).cuda()
        x3 = torch.from_numpy(np.load(x3_name)).unsqueeze(0).unsqueeze(0).cuda()

        predict_label = tripleNet_LSTM_model(x1,x2,x3).detach().cpu()
        result_score_dict[token] = predict_label[0][0]


In [17]:
print(len(my_test_data))
print(len(result_score_dict))
# for key in result_score_dict:
#     print(key,result_score_dict[key])

6000
6000


In [20]:
from __future__ import division
testLength = 1000

def get_result_key(index,x1,x2,x3):
    return "song"+str(index)+"_"+str(x1)+","+"song"+str(index)+"_"+str(x2)+","+"song"+str(index)+"_"+str(x3)

def get_result_score(index):
    scoreList = []
    score_123 = result_score_dict[get_result_key(index,1,2,3)] 
    score_132 = result_score_dict[get_result_key(index,1,3,2)] 
    score_213 = result_score_dict[get_result_key(index,2,1,3)] 
    score_231 = result_score_dict[get_result_key(index,2,3,1)] 
    score_312 = result_score_dict[get_result_key(index,3,1,2)] 
    score_321 = result_score_dict[get_result_key(index,3,2,1)] 
    scoreList.append(score_123)
    scoreList.append(score_132)
    scoreList.append(score_213)
    scoreList.append(score_231)
    scoreList.append(score_312)
    scoreList.append(score_321)
    import operator
    index, value = max(enumerate(scoreList), key=operator.itemgetter(1))
    return index,value

def cal_accuracy(index):
    if(index==0):
        return 2,True
    elif(index==1):
        return 0,False
    elif(index==2):
        return 0,False
    elif(index==3):
        return 1,False
    elif(index==4):
        return 1,False
    elif(index==5):
        return 0,False
    else:
        return -1,False 

GA = [0,0,0,0,0,0]
PA = 0.0
for i in range(6000,7000):
    index, value = get_result_score(i)
#     print(index,value)
    pair,_ = cal_accuracy(index)
    PA += pair/3
    if(index == 0):
        GA[0] += 1
    if(index == 1):
        GA[1] += 1
    if(index == 2):
        GA[2] += 1
    if(index == 3):
        GA[3] += 1
    if(index == 4):
        GA[4] += 1
    if(index == 5):
        GA[5] += 1

#GA[0] is "123"
#GA[1,2,,3,4,5] is "132,213,231,312,321"
print("Accuracy:",GA[0]/testLength)

# for i in range(6):
#     print("Accuracy:",GA[i]/testLength)

Accuracy: 0.758
